In [47]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from numpy.linalg import multi_dot
from scipy.stats import expon


In [48]:
#Reading Regressors.txt
df = pd.read_csv("Regressors.txt", header = None)
df.head

<bound method NDFrame.head of     0         1        2        3  ...        5         6         7        8
0   1 -0.637280 -2.53690  1.00570  ...  0.74296 -0.900550 -0.385430 -1.01310
1   1  0.329080 -0.87360  1.51210  ...  0.49154  0.862480 -0.053105  1.00120
2   1  0.369910 -1.05260 -0.76142  ... -0.21182 -0.064187 -0.133520  0.62344
3   1 -0.536420  1.20020  0.31715  ... -0.29137 -1.168800  0.010371 -0.55335
4   1  0.750650 -0.23990 -0.48102  ...  0.84527 -0.965980 -0.795350  0.65267
.. ..       ...      ...      ...  ...      ...       ...       ...      ...
95  1  0.778860 -0.77748  0.35712  ... -0.35897 -1.420100  0.853770  0.42224
96  1 -0.735240 -1.16190 -1.21380  ...  0.24747 -1.406300 -1.143000 -1.37200
97  1 -0.974290  1.54520 -0.12210  ...  2.66050  0.915620 -1.017000  0.96036
98  1 -0.018074 -1.40830  0.87636  ... -0.91976  0.754050  0.013707 -0.62020
99  1  1.663100 -0.26624  0.07666  ... -3.03220 -0.145520  0.448210 -0.51953

[100 rows x 9 columns]>

In [49]:
#Reading Observables.txt
df_y = pd.read_csv("Observables.txt", header = None)
df_y.head()
y_array = np.array(df_y)


In [50]:
#(I-X(X'X)^-1-X')y
m = np.array(df)
m_trans = m.transpose()
first_mult = multi_dot([m_trans, m])
inverse = np.linalg.inv(first_mult) 
sec_mult = multi_dot([m, inverse, m_trans])
sec_mult.shape
identity_matrix = np.identity(100)
identity_matrix
sub =np.subtract(identity_matrix, sec_mult)
epsilon = multi_dot([sub, y_array])
epsilon


array([[-1.94852220e-01],
       [ 1.72042305e+00],
       [ 2.63451544e+00],
       [-8.03040879e-01],
       [-2.92266812e-01],
       [ 8.25102712e-01],
       [ 2.81847042e+00],
       [-2.05363896e+00],
       [-5.40266864e-01],
       [ 2.87600740e-01],
       [ 1.75601661e+00],
       [ 1.30787704e+00],
       [ 6.73171122e-01],
       [ 3.79998055e-01],
       [ 8.58296227e-01],
       [ 4.47623874e-01],
       [-8.92123116e-01],
       [-1.20760758e+00],
       [ 1.82435791e+00],
       [ 2.00439531e+00],
       [-1.24199264e+00],
       [-3.63540721e-01],
       [-1.64964896e+00],
       [ 1.48863152e+00],
       [-3.16597254e-01],
       [-1.36671477e-02],
       [-8.24819159e-02],
       [-1.00112155e+00],
       [ 7.21221941e-01],
       [-5.95253364e-01],
       [ 1.12453599e-01],
       [ 3.46162563e-01],
       [-5.25655727e-01],
       [-1.33624176e+00],
       [ 8.53864143e-01],
       [-7.45330655e-01],
       [-1.03828372e+00],
       [-4.01300431e-01],
       [-8.0

In [51]:
# sum (e(t) - e(t-1))^2
eps_diff = np.diff(epsilon, n=1, axis=0,)
eps_sqr = np.square(eps_diff)
sum1 = np.sum(eps_sqr, axis=0)
sum1


array([192.91561707])

In [52]:
eps_den = np.square(epsilon[:-1])
sum2 = np.sum(eps_den)
sum2

107.72268508774314

In [53]:
test_statistic = list(sum1/sum2)
test_statistic

[1.7908541447664765]

In [54]:
#Monte Carlo Test
list_tstats = [test_statistic[0]]
for n in range(9999):
  #I made it 100 instead of 1000 because our sample size 100 (100 data points) and the size affects the measurement Durbin-Watson statistics, now results make sense
  ran_exp = np.random.standard_normal(size = 100)
  eps_diff = np.diff(ran_exp, n=1, axis=0,)
  eps_sqr1 = np.square(eps_diff)
  ep_sqr = np.square(ran_exp)
  suma = np.sum(eps_sqr1)
  sumb = np.sum(ep_sqr)
  t_stat = suma/sumb
  list_tstats.append(t_stat)




In [55]:
#rank the simulated data
list_tstats.sort()
list_tstats[0:5]
list_tstats[-5:]

[2.625426703327039,
 2.6333814786681504,
 2.640701570202458,
 2.6446999268896625,
 2.679834526695033]

In [56]:
k1 = int((9999+1)*(0.1/2) + 1)
k2 = int((1-0.1/2)*(9999+1))

In [58]:
rej_c2 = list_tstats[k2-1]
rej_c2

2.301515126522506

In [59]:
print(rej_c1," ",rej_c2)
print(test_statistic)

1.6554690338114069   2.301515126522506
[1.7908541447664765]


In [60]:
def check(list, val):
  num = 0
  for n in list:
    if n <= val:
      num = num+1
    else:
      num = num
  return num


In [61]:
value = check(list_tstats, test_statistic[0])
value

1714

In [62]:
def check2(list, val):
  num = 0
  for n in list:
    if n >= val:
      num = num+1
    else:
      num = num
  return num

In [63]:
value2 = check2(list_tstats, test_statistic[0])
value2

8287

In [64]:
#Both denominators must be 10000
p_value = 2 *min([value/10000, value2/10000])
p_value

0.3428

## Part C

In [65]:
#Reading True_null.txt
df_tn = pd.read_csv("True_null.txt", header = None)
df_tn.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9960,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,-0.62792,-0.24685,0.257740,-1.66090,-0.47635,-1.11590,-2.08230,-1.80800,1.17990,-1.11960,0.48175,-0.67854,-1.27570,0.037071,1.05500,-2.37350,0.037833,-0.39659,1.35980,-0.40209,0.913340,-1.280600,0.98981,0.50423,-0.98986,-0.15408,-1.11310,-2.18270,0.42593,-2.516600,0.21081,-1.76250,-0.62526,0.56285,0.420430,0.550320,-1.272000,0.214790,-1.52930,-1.78570,...,0.401270,0.65703,-1.57650,0.26412,-0.55309,-0.23669,-0.26533,-0.88291,-1.86140,-0.478620,-0.046582,0.21400,1.21700,-1.42370,-0.897930,-1.841700,-0.680820,1.31460,0.021599,-2.06930,-0.306510,-1.09370,-2.77060,-1.43940,-0.53657,-1.03330,-1.383500,-1.19920,0.321470,0.41930,-0.574400,-0.65947,1.61390,-0.897910,-0.52626,-1.36490,-1.274500,-2.260300,-0.91631,-0.303750
1,1.38720,0.58790,-1.174900,0.13679,-1.84960,-0.59578,-0.26431,1.07710,1.01380,0.48220,-0.31815,-1.24950,-0.40086,-0.765230,-0.34760,0.83993,-0.992750,0.23665,0.52186,-0.24556,-0.052441,-0.061019,-0.71545,-0.15051,-0.96491,-1.10580,-0.31657,0.33082,-0.80673,-0.027455,-0.91155,0.97031,1.46700,-1.66200,-1.556500,-1.576600,0.374200,-1.006700,0.78580,-0.90509,...,0.146970,-0.45322,1.22510,1.78850,-0.52633,0.52794,0.57792,-0.37415,1.77960,0.951180,-0.826920,-0.67055,-0.25470,-2.12020,0.091345,-0.074959,-0.363810,0.20969,-1.763700,-0.65442,-0.235580,-0.67057,-0.36307,-1.16640,-1.05440,0.17899,-1.453100,-0.89240,-0.459560,-1.67450,0.661720,-1.48890,0.40876,0.157520,-1.18770,-0.91731,-1.830700,-0.062291,-0.70922,0.787200
2,2.20110,-0.43099,-0.026149,0.44379,1.19560,0.42235,-0.30351,-1.13260,0.22418,0.49966,-0.70172,-2.37990,0.33363,0.411320,0.84880,-2.07210,0.663290,0.12283,1.69490,-0.20957,0.184540,1.031500,0.33449,-1.91550,-1.29580,-2.01410,-0.47632,1.17550,-1.18170,0.982100,1.61230,1.29550,-0.94170,-0.57267,0.032398,0.093877,-0.021064,1.600700,-0.42249,-1.79090,...,-0.978810,1.20540,-0.21347,-1.45890,-0.46411,-3.03390,-1.43240,-0.92275,0.90932,-0.012546,-0.190940,-1.36340,2.10560,-0.24746,-0.882770,-0.208110,1.548500,-0.84247,0.650140,-0.10475,-2.394900,0.75807,-1.40760,1.20900,0.60336,1.03520,-1.355200,0.38104,-1.006100,0.12119,-0.037405,0.77088,-1.06980,2.186100,1.02760,0.97215,0.804750,-0.799210,0.77229,1.481000
3,-1.19320,1.24200,0.258260,1.48780,0.69618,-1.04460,-0.64679,-1.42940,-0.11541,-1.07190,-0.88992,-2.05290,0.61045,0.685880,-0.82170,-1.39460,-0.340230,0.11472,-1.79830,0.51816,-1.227400,-0.627860,-1.98050,-0.53659,0.82022,-1.28550,-0.18785,-0.66317,-0.82253,-0.062926,1.06380,0.73375,-2.54740,-2.32490,-1.091100,-1.546200,-0.596780,-0.219940,-2.84450,0.26861,...,-0.048599,-1.95640,-0.82907,-0.72639,1.22060,1.49040,-1.99640,-0.90021,-0.09871,-0.511030,2.120500,-1.58680,-0.52104,-0.39895,-1.222600,-1.019800,1.670700,-0.17398,-0.785420,2.39570,-0.085546,-1.66930,2.60030,-0.15321,-0.72681,0.41847,-0.027018,-0.72439,-0.063239,0.24325,-1.433600,0.18135,0.51026,0.086533,-1.37540,-1.14550,-2.642300,-0.384730,0.77222,-0.008419
4,-0.74989,1.76020,-0.442110,-1.68450,0.13206,-1.10050,-2.18150,-0.65405,-0.97755,-0.49721,0.47525,0.29390,-0.77628,1.249700,-0.23502,-0.62788,-2.882500,-0.87602,1.02540,-2.18670,-0.611910,-1.241800,0.35059,-1.47040,-1.58930,-1.01070,0.53050,-1.41770,-2.09830,0.298590,0.22370,-0.12858,-2.37410,-1.02550,-0.131160,0.454360,0.539740,-0.046699,2.09000,-0.36788,...,0.159360,-1.45240,-0.13816,-1.50600,-0.66690,-1.41060,-0.65905,-0.38332,0.16310,1.111200,0.056765,0.67707,-0.25127,0.32764,0.546360,-1.972300,0.033561,0.65132,-0.804400,-0.70066,-1.395700,-0.80951,-0.79878,0.22992,-0.55406,-1.42200,-0.790870,-0.37103,-1.973000,-0.61356,-0.279060,-0.66515,0.21881,-0.388650,-1.32840,0.28258,0.021629,-2.858200,-1.91630,-0.943880


In [66]:
def pval(list,val,size):
    return  2*min([check(list,val)/size+1, check2(list,val)/size+1])
    
probs = np.zeros(10000)


#New Solution

#Rejection check for N columns 
for i in range(10000):
    y_array = np.array(df_tn[i])
    epsilon = multi_dot([sub, y_array])
    
    eps_diff = np.diff(epsilon, n=1, axis=0,)
    eps_sqr = np.square(eps_diff)
    sum1 = np.sum(eps_sqr, axis=0)
    
    eps_den = np.square(epsilon)
    sum2 = np.sum(eps_den)
    
    test_statistic = sum1/sum2
    if(test_statistic<rej_c1 or test_statistic>rej_c2):
        probs[i] = 1
    


#Probability of rejection
alpha = np.mean(probs)
    

In [67]:
print(alpha) 

0.0995


## Part E

In [68]:
df_fn = pd.read_csv("False_null.txt", header = None)
df_fn.head


<bound method NDFrame.head of         0        1        2        3     ...     9996     9997     9998      9999
0  -0.712300 -0.33122  0.17336 -1.74520  ... -1.35880 -2.34460 -1.00070 -0.388120
1   1.244700  0.52166 -1.14020 -0.21226  ... -2.10250 -0.53122 -0.90936  0.709570
2   2.450000 -0.32665 -0.25420  0.40134  ...  0.38425 -0.90545  0.59042  1.622900
3  -0.703170  1.17660  0.20742  1.56810  ... -2.56550 -0.56582  0.89030  0.316160
4  -0.890530  1.99550 -0.40062 -1.37080  ... -0.49146 -2.97140 -1.73820 -0.880650
..       ...      ...      ...      ...  ...      ...      ...      ...       ...
95 -0.062718  0.51397  0.41479 -1.22140  ... -0.17607 -0.59603  0.84981  2.639000
96 -1.008500  0.34215 -2.01440 -0.21421  ... -3.16940 -2.71010 -1.71630 -0.174510
97 -1.679800 -0.56710 -1.50140 -2.56090  ... -1.00990 -1.68450 -0.59865 -2.751700
98  0.205610  1.30220 -0.47946 -1.93270  ... -0.90779 -0.96402  0.16431 -0.093105
99  0.101860 -1.64230 -0.56062 -1.93880  ...  1.37600 -1.54740 -0.19

In [71]:
probs = np.zeros(10000)
true_coverage = np.zeros(10000)

#New Solution

#Coverage and Rejection check for N columns
for i in range(10000):
    y_array = np.array(df_fn[i])
    epsilon = multi_dot([sub, y_array])
    
    eps_diff = np.diff(epsilon, n=1, axis=0,)
    eps_sqr = np.square(eps_diff)
    sum1 = np.sum(eps_sqr, axis=0)
    
    eps_den = np.square(epsilon)
    sum2 = np.sum(eps_den)
    
    test_statistic = sum1/sum2
    probs[i] = (test_statistic >= rej_c1 and test_statistic <= rej_c2)
    true_coverage[i] = ((rej_c1-test_statistic)/2 <=  0.2 and 0.2  <=(rej_c2-test_statistic)/2)
    
    

#type-2 error, (not rejecting null while it is false)
beta = np.mean(probs)


In [72]:
# power of the test = (1 - beta)
print(1-beta)
print(np.mean(true_coverage))


0.5206
0.8881


In [40]:
#Old Solution

#size = 99
#
#for i in range(10000):   
#    y_array = np.array(df_tn[i])
#    epsilon = multi_dot([sub, y_array])
#    eps_diff = np.diff(epsilon, n=1, axis=0,)
#    eps_sqr = np.square(eps_diff)
#    sum1 = np.sum(eps_sqr, axis=0)
#    eps_den = np.square(epsilon)
#    sum2 = np.sum(eps_den)
#    test_statistic = sum1/sum2
#    list_tstats = [test_statistic]
#    
#    for n in range(size):
#        ran_exp = np.random.standard_normal(size = 100)
#        eps_diff = np.diff(ran_exp, n=1, axis=0,)
#        eps_sqr1 = np.square(eps_diff)
#        ep_sqr = np.square(ran_exp)
#        suma = np.sum(eps_sqr1)
#        sumb = np.sum(ep_sqr)
#        t_stat = suma/sumb
#        list_tstats.append(t_stat)
#        
#    k1 = int((size+1)*(0.1/2) + 1)
#    k2 = int((1-0.1/2)*(size+1))
#    rej_c1 = list_tstats[k1-1]
#    rej_c2 = list_tstats[k2-1]
#    
#    if (test_statistic<rej_c1 or test_statistic>rej_c2):
#        probs[i] = 1
        
        
    
#Old Solution

#size = 99
#
#for i in range(10000):   
#    y_array = np.array(df_fn[i])
#    epsilon = multi_dot([sub, y_array])
#    eps_diff = np.diff(epsilon, n=1, axis=0,)
#    eps_sqr = np.square(eps_diff)
#    sum1 = np.sum(eps_sqr, axis=0)
#    eps_den = np.square(epsilon)
#    sum2 = np.sum(eps_den)
#    test_statistic = sum1/sum2
#    list_tstats = [test_statistic]
#    
#    for n in range(size):
#        ran_exp = np.random.standard_normal(size = 100)
#        eps_diff = np.diff(ran_exp, n=1, axis=0,)
#        eps_sqr1 = np.square(eps_diff)
#        ep_sqr = np.square(ran_exp)
#        suma = np.sum(eps_sqr1)
#        sumb = np.sum(ep_sqr)
#        t_stat = suma/sumb
#        list_tstats.append(t_stat)
#        
#    k1 = int((size+1)*(0.1/2) + 1)
#    k2 = int((1-0.1/2)*(size+1))
#    rej_c1 = list_tstats[k1-1]
#    rej_c2 = list_tstats[k2-1]
#    
#    if (1-rej_c2/2 <= 0.2 and 0.2 <=1-rej_c1/2):
#        true_coverage[i] = 1
#    if (test_statistic>rej_c1 and test_statistic<rej_c2):
#        probs[i] = 1


